The collaborative filter approach focuses on finding users who have given similar ratings to the same books, thus creating a link between users, to whom will be suggested books that were reviewed in a positive way. In this way, we look for associations between users, not between books.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

2023-04-03 19:45:25.023179: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
rating = pd.read_csv('FoodRatings2.csv', sep=';', error_bad_lines=False, encoding="latin-1")
user = pd.read_csv('UserID.csv', sep=';', error_bad_lines=False, encoding="latin-1")
food = pd.read_csv('Food.csv', sep=';', error_bad_lines=False, encoding="latin-1")
food_rating = pd.merge(rating, food, on='foodid')
cols = ['UserID', 'FoodID', 'Rating', 'Food']
##food_rating.drop(cols, axis=1, inplace=True)
food_rating

/var/folders/13/x078bl690h73w_0kzkf15bx80000gn/T/ipykernel_5628/2323556517.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  rating = pd.read_csv('FoodRatings2.csv', sep=';', error_bad_lines=False, encoding="latin-1")
/var/folders/13/x078bl690h73w_0kzkf15bx80000gn/T/ipykernel_5628/2323556517.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  user = pd.read_csv('UserID.csv', sep=';', error_bad_lines=False, encoding="latin-1")
/var/folders/13/x078bl690h73w_0kzkf15bx80000gn/T/ipykernel_5628/2323556517.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  food = pd.read_csv('Food.csv', sep=';', error_bad_lines=False, encoding="latin-1")


,userid,foodid,rating,food
0,1,16,3,Char Kway Teow
1,16,16,3,Char Kway Teow
2,26,16,4,Char Kway Teow
3,29,16,2,Char Kway Teow
4,46,16,1,Char Kway Teow
...,...,...,...,...
7436,15,82,3,Teppanyaki
7437,55,82,4,Teppanyaki
7438,47,82,3,Teppanyaki
7439,51,82,2,Teppanyaki


In [9]:
rating_count = (food_rating.
     groupby(by = ['food'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'RatingCount_Food'})
     [['food', 'RatingCount_Food']])
rating_count

,food,RatingCount_Food
0,Assam Pedas,40
1,Ayam Penyet,74
2,Bak Kut Teh,80
3,Bak chor mee,76
4,Bakso,98
...,...,...
84,rawon,99
85,soup kambing,93
86,takoyaki,83
87,text,78


In [10]:
threshold = 10
rating_count = rating_count.query('RatingCount_Food >= @threshold')
rating_count.head()

,food,RatingCount_Food
0,Assam Pedas,40
1,Ayam Penyet,74
2,Bak Kut Teh,80
3,Bak chor mee,76
4,Bakso,98


In [11]:
user_rating = pd.merge(rating_count, food_rating, left_on='food', right_on='food', how='left')

In [12]:
user_rating.head()

,food,RatingCount_Food,userid,foodid,rating
0,Assam Pedas,40,16,1,3
1,Assam Pedas,40,24,1,5
2,Assam Pedas,40,56,1,1
3,Assam Pedas,40,22,1,4
4,Assam Pedas,40,57,1,3


In [13]:
user_count = (user_rating.
     groupby(by = ['userid'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'RatingCount_user'})
     [['userid', 'RatingCount_user']]
    )
user_count

,userid,RatingCount_user
0,1,33
1,2,86
2,3,69
3,4,68
4,5,69
...,...,...
95,96,78
96,97,79
97,98,86
98,99,89


In [30]:
threshold = 10
user_count = user_count.query('RatingCount_user >= @threshold')
user_count

,userid,RatingCount_user
0,1,33
1,2,86
2,3,69
3,4,68
4,5,69
...,...,...
95,96,78
96,97,79
97,98,86
98,99,89


In [31]:
combined = user_rating.merge(user_count, left_on = 'userid', right_on = 'userid', how = 'inner')

In [32]:
combined

,food,RatingCount_Food,userid,foodid,rating,RatingCount_user
0,Assam Pedas,40,16,1,3,77
1,BanMian,90,16,7,4,77
2,BanMian,90,16,7,4,77
3,BanMian,90,16,7,4,77
4,BanMian,90,16,7,1,77
...,...,...,...,...,...,...
7436,takoyaki,83,1,80,2,33
7437,tom yum,78,1,85,4,33
7438,tom yum,78,1,85,4,33
7439,tom yum,78,1,85,4,33


In [33]:
combined.shape

(7441, 6)

In [34]:
print('Number of unique Food: ', combined['food'].nunique())
print('Number of unique Users: ', combined['userid'].nunique())

Number of unique Food:  89
Number of unique Users:  100


Normalize the ratings.

In [19]:
scaler = MinMaxScaler()
combined['rating'] = combined['rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(combined['rating'].values.reshape(-1,1)))
combined['rating'] = rating_scaled

Abd build the user book matrix.

In [20]:
combined = combined.drop_duplicates(['userid', 'food'])
user_food_matrix = combined.pivot(index='userid', columns='food', values='rating')
user_food_matrix.fillna(0, inplace=True)

users = user_food_matrix.index.tolist()
books = user_food_matrix.columns.tolist()

user_food_matrix = user_food_matrix.values

,food,RatingCount_Food,userid,foodid,rating,RatingCount_user
0,Assam Pedas,40,16,1,3,77
1,BanMian,90,16,7,4,77
2,BanMian,90,16,7,4,77
3,BanMian,90,16,7,4,77
4,BanMian,90,16,7,1,77
...,...,...,...,...,...,...
7436,takoyaki,83,1,80,2,33
7437,tom yum,78,1,85,4,33
7438,tom yum,78,1,85,4,33
7439,tom yum,78,1,85,4,33


tf.placeholder only available in v1, so we have to work around. 

In [21]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


We will initialize the TensorFlow placeholder. Then, weights and biases are randomly initialized, the following code are taken from the book: Python Machine Learning Cook Book - Second Edition

In [22]:
num_input = combined['food'].nunique()
num_hidden_1 = 10
num_hidden_2 = 5

X = tf.placeholder(tf.float64, [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1], dtype=tf.float64)),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2], dtype=tf.float64)),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1], dtype=tf.float64)),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input], dtype=tf.float64)),
}

biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2], dtype=tf.float64)),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'decoder_b2': tf.Variable(tf.random_normal([num_input], dtype=tf.float64)),
}

Now, we can build the encoder and decoder model, as follows:

In [23]:
def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))
    return layer_2

def decoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']), biases['decoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']), biases['decoder_b2']))
    return layer_2

We will construct the model and the predictions

In [24]:
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

y_pred = decoder_op

y_true = X

define loss function and optimizer, and minimize the squared error, and define the evaluation metrics

In [25]:
loss = tf.losses.mean_squared_error(y_true, y_pred)
optimizer = tf.train.RMSPropOptimizer(0.03).minimize(loss)
eval_x = tf.placeholder(tf.int32, )
eval_y = tf.placeholder(tf.int32, )
pre, pre_op = tf.metrics.precision(labels=eval_x, predictions=eval_y)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Initialize the variables. Because TensorFlow uses computational graphs for its operations, placeholders and variables must be initialized.

In [26]:
init = tf.global_variables_initializer()
local_init = tf.local_variables_initializer()
pred_data = pd.DataFrame()

We can finally start to train our model.

We split training data into batches, and we feed the network with them.

We train our model with vectors of user ratings, each vector represents a user and each column a book, and entries are ratings that the user gave to books. 

After a few trials, I discovered that training model for 5 epochs with a batch size of 10 would be consum enough memory. This means that the entire training set will feed our neural network 20 times, every time using 50 users.

In [27]:
with tf.Session() as session:
    epochs = 100
    batch_size = 35

    session.run(init)
    session.run(local_init)

    num_batches = int(user_food_matrix.shape[0] / batch_size)
    user_food_matrix = np.array_split(user_food_matrix, num_batches)
    
    for i in range(epochs):

        avg_cost = 0
        for batch in user_food_matrix:
            _, l = session.run([optimizer, loss], feed_dict={X: batch})
            avg_cost += l

        avg_cost /= num_batches

        print("epoch: {} Loss: {}".format(i + 1, avg_cost))

    user_food_matrix = np.concatenate(user_food_matrix, axis=0)

    preds = session.run(decoder_op, feed_dict={X: user_food_matrix})

    pred_data = pred_data.append(pd.DataFrame(preds))

    pred_data = pred_data.stack().reset_index(name='rating')
    pred_data.columns = ['userid', 'food', 'rating']
    pred_data['userid'] = pred_data['userid'].map(lambda value: users[value])
    pred_data['food'] = pred_data['food'].map(lambda value: books[value])
    
    keys = ['userid', 'food']
    index_1 = pred_data.set_index(keys).index
    index_2 = combined.set_index(keys).index

    top_ten_ranked = pred_data[~index_1.isin(index_2)]
    top_ten_ranked = top_ten_ranked.sort_values(['userid', 'rating'], ascending=[True, False])
    top_ten_ranked = top_ten_ranked.groupby('userid').head(10)

epoch: 1 Loss: 0.27454185485839844
epoch: 2 Loss: 0.2744889557361603
epoch: 3 Loss: 0.27443014085292816
epoch: 4 Loss: 0.2743648588657379
epoch: 5 Loss: 0.27429236471652985
epoch: 6 Loss: 0.2742117941379547
epoch: 7 Loss: 0.27412234246730804
epoch: 8 Loss: 0.27402299642562866
epoch: 9 Loss: 0.273912712931633
epoch: 10 Loss: 0.27379022538661957
epoch: 11 Loss: 0.2736542224884033
epoch: 12 Loss: 0.27350327372550964
epoch: 13 Loss: 0.27333568036556244
epoch: 14 Loss: 0.27314962446689606
epoch: 15 Loss: 0.2729432284832001
epoch: 16 Loss: 0.27271412312984467
epoch: 17 Loss: 0.2724599391222
epoch: 18 Loss: 0.2721780389547348
epoch: 19 Loss: 0.2718653678894043
epoch: 20 Loss: 0.27151869237422943
epoch: 21 Loss: 0.2711343914270401
epoch: 22 Loss: 0.27070853114128113
epoch: 23 Loss: 0.2702367454767227
epoch: 24 Loss: 0.2697143107652664
epoch: 25 Loss: 0.26913607120513916
epoch: 26 Loss: 0.2684963643550873
epoch: 27 Loss: 0.2677890658378601
epoch: 28 Loss: 0.267007514834404
epoch: 29 Loss: 0.266

/var/folders/13/x078bl690h73w_0kzkf15bx80000gn/T/ipykernel_5628/655726344.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pred_data = pred_data.append(pd.DataFrame(preds))


In [28]:
top_ten_ranked.loc[top_ten_ranked['userid'] == 70]

,userid,food,rating
6203,70,Satay,0.423272
6198,70,Red curry,0.377414
6183,70,Mee goreng,0.377060
6184,70,Mee siam,0.375292
6229,70,tom yum,0.336617
6173,70,Hainanese chicken rice,0.336518
6220,70,katong Laksa,0.330029
6175,70,Har cheong gai,0.321340
6201,70,Roti john,0.319191
6146,70,BanMian,0.316567


In [25]:
book_rating.loc[book_rating['User-ID'] == 278582].sort_values(by=['Book-Rating'], ascending=False)

,User-ID,ISBN,Book-Rating,Book-Title
174885,278582,0226848620,10,Chinese Bell Murders (Judge Dee Mysteries)
176582,278582,157566254X,10,"Skin Deep, Blood Red"
40008,278582,0441478123,10,The Left Hand of Darkness (Remembering Tomorrow)
174861,278582,0061044725,10,Search the Shadows
58156,278582,0451202503,10,The Songcatcher: A Ballad Novel
64570,278582,1400034779,10,The No. 1 Ladies' Detective Agency (Today Show...
175958,278582,0345350499,10,The Mists of Avalon
176314,278582,0449223558,9,Murdering Mr. Monti: A Merry Little Tale of Se...
174877,278582,0140277471,9,Blanche Cleans Up
176438,278582,0515136557,8,The Cat Who Brought Down the House
